## GEE Map

This tutorial was developed to teach you all how to use geemap which is a python library developed to use Google Earth Engine in python. I modified this [tutorial](https://geemap.org/notebooks/39_timelapse/) to teach you how to create a landsat timelapse. 

The goals of this tutorial are as follows:
* Link your jupyter notebook to GEE
* Load landsat image collections
* Time series GIF

In [1]:
### 
import ipywidgets as widgets###creating our own widget to build the timeseries
import geemap
import os
import ee

In [2]:
###let's connect you to GEE
Map = geemap.Map()

In [3]:
###This map will display your timelapse :)
'''
First, zoom in to a region of interest and draw a polygon over where ever you want.
Make sure the polygon isn't huge though (so, don't draw one over the entirety of Colorado.
Your memory can't handle that many images.

For this tutorial, i'll be using Sandy Hook NJ. 
'''
Map.add_basemap('HYBRID')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [4]:
style = {'description_width': 'initial'}
title = widgets.Text(
    description='Title:', value='Landsat Timelapse', width=200, style=style
)

'''
To make our lives easier, we will only use 2 band combos:
RGB and NIR/R/G. One is a true color image, another is a 
false color image. 

'''

bands = widgets.Dropdown(
    description='Select RGB Combo:',
    options=[
        'Red/Green/Blue',
        'NIR/Red/Green',
    ],
    value='Red/Green/Blue',
    style=style,
)

hbox1 = widgets.HBox([title, bands])
hbox1

In [5]:
speed = widgets.IntSlider(
    description='  Frames per second:',
    tooltip='Frames per second:',
    value=10,
    min=1,
    max=30,
    style=style,
)

cloud = widgets.Checkbox(
    value=True, description='Apply fmask (remove clouds, shadows, snow)', style=style
)

hbox2 = widgets.HBox([speed, cloud])
hbox2

In [6]:
start_year = widgets.IntSlider(
    description='Start Year:', value=1984, min=1984, max=2020, style=style
)
end_year = widgets.IntSlider(
    description='End Year:', value=2020, min=1984, max=2020, style=style
)
start_month = widgets.IntSlider(
    description='Start Month:', value=5, min=1, max=12, style=style
)
end_month = widgets.IntSlider(
    description='End Month:', value=10, min=1, max=12, style=style
)
hbox3 = widgets.HBox([start_year, end_year, start_month, end_month])
hbox3

In [7]:
font_size = widgets.IntSlider(
    description='Font size:', value=30, min=10, max=50, style=style
)

font_color = widgets.ColorPicker(
    concise=False, description='Font color:', value='white', style=style
)

progress_bar_color = widgets.ColorPicker(
    concise=False, description='Progress bar color:', value='blue', style=style
)

hbox4 = widgets.HBox([font_size, font_color, progress_bar_color])
hbox4

In [8]:
create_gif = widgets.Button(
    description='Create timelapse',
    button_style='primary',
    tooltip='Click to create timelapse',
    style=style,
)

download_gif = widgets.Button(
    description='Download GIF',
    button_style='primary',
    tooltip='Click to download timelapse',
    disabled=False,
    style=style,
)

output = widgets.Output()

hbox5 = widgets.HBox([create_gif])
hbox5

In [9]:
def submit_clicked(b):
    with output:
        output.clear_output()
        if start_year.value > end_year.value:
            print('The end year must be great than the start year.')
            return
        if start_month.value > end_month.value:
            print('The end month must be great than the start month.')
            return
        if start_year.value == end_year.value:
            add_progress_bar = False
        else:
            add_progress_bar = True

        start_date = str(start_month.value).zfill(2) + '-01'
        end_date = str(end_month.value).zfill(2) + '-30'

        print('Computing...')

        Map.add_landsat_ts_gif(
            roi=Map.user_roi,
            label=title.value,
            start_year=start_year.value,
            end_year=end_year.value,
            start_date=start_date,
            end_date=end_date,
            bands=bands.value.split('/'),
            font_color=font_color.value,
            frames_per_second=speed.value,
            font_size=font_size.value,
            add_progress_bar=add_progress_bar,
            progress_bar_color=progress_bar_color.value,
            download=True,
            apply_fmask=cloud.value,
        )


create_gif.on_click(submit_clicked)

In [10]:
output

Output(outputs=({'name': 'stdout', 'text': 'Computing...\nGenerating URL...\n', 'output_type': 'stream'},))